# Numerical Computing in Python

(C) 2016 [Steve Phelps](http://sphelps.net)

_Modifications to convert to Python 3, Rebecca Lowdon (Feb 2019)._

# Overview

- Floating-point representation
- Arrays and Matrices with `numpy`
- Basic plotting with `matplotlib`
- Pseudo-random variates with `numpy.random`

# Representing continuous values

- Digital computers are inherently *discrete*.

- Real numbers $x \in R$ cannot always be represented exactly in a digital computer.

- They are stored in a format called *floating-point*.

- [IEEE Standard 754](http://steve.hollasch.net/cgindex/coding/ieeefloat.html) specifies a universal format across different implementations.  
    - As always there are deviations from the standard.
    
- There are two standard sizes of floating-point numbers: 32-bit and 64-bit.

- 64-bit numbers are called *double precision*, are sometimes called *double* values.

- IEEE floating-point calculations are performed in *hardware* on modern computers. 

- How can we represent aribitrary real values using only 32 bits?
   

# Fixed-point verses floating-point

* **CoderGirl**: Please read through the first three sections of this [primer on floating point numbers](http://steve.hollasch.net/cgindex/coding/ieeefloat.html). 
    - What Are Floating Point Numbers?
    - Storage Layout
    - Ranges of Floating-Point Number

# Scientific Notation

- Humans also use a form of floating-point representation.

- In [Scientific notation](https://en.wikipedia.org/wiki/Scientific_notation), all numbers are written
in the form $m \times 10^n$.

- When represented in ASCII, we abbreviate this `<m>e<n>`, for example `6.72e+11` = $6.72 \times 10^{11}$.

- The integer $m$ is called the *significand* or *mantissa*. 

- The integer $n$ is called the *exponent*.

- The integer $10$ is the *base*.


# Scientific Notation in Python

- Use `format()` to implement Scientific notation for floating-point numbers:


In [ ]:
print('{:.2e}'.format(672000000000.0))

- Note that internally, the value is not *represented* exactly like this.  

- Scientific notation is a convention for writing or rendering numbers, *not* representing them digitally. 

# Effective floating-point range

| Format | Binary | Decimal |
| ------ | ------ | ------- |
| single | $\pm (2-2^{-23}) \times 2^{127}$ | $\approx \pm 10^{38.53}$ |
| double | $\pm (2-2^{-52}) \times 2^{1023}$ | $\approx \pm 10^{308.25}$ |

    

# 

# Representing Zero

- Zero cannot be represented straightforwardly because we assume that all mantissa values start with the digit 1.

- Zero is stored as a special-case, by setting mantissa *and* exponent both to zero.

- The sign-bit can either be set or unset, so there are distinct positive and negative representations of zero.


# Zero in Python

In [ ]:
x = +0.0
x

In [ ]:
y = -0.0
y

- However, these are considered equal:

In [ ]:
x == y

# Infinity

- Positive overflow results in a special value of infinity (in Python `inf`).

- This is stored with an exponent consiting of all 1s, and a mantissa of all 0s.

- The sign-bit allows us to differentiate between negative and positive overflow: $-\infty$ and $+\infty$.

- This allows us to carry on calculating past an overflow event.

# Infinity in Python

In [ ]:
x = 1e300 * 1e100
x

In [ ]:
x = x + 1
x

# Negative infinity in Python

In [ ]:
x > 0

In [ ]:
y = -x
y

In [ ]:
y < x

# Not A Number (NaN)

- Some mathematical operations on real numbers do not map onto real numbers.

- These results are represented using the special value to `NaN` which represents "not a (real) number".

- `NaN` is represented by an exponent of all 1s, and a non-zero mantissa.


# `NaN` in Python



In [ ]:
from numpy import sqrt, inf, isnan, nan
x = sqrt(-1)
x

In [ ]:
y = inf - inf
y

# Comparing `nan` values in Python

- Beware of comparing `nan` values

In [ ]:
x == y

- To test whether a value is `nan` use the `isnan` function:

In [ ]:
isnan(x)

# `NaN` is not the same as `None`

- `None` represents a *missing* value.

- `NaN` represents an *invalid* floating-point value.

- These are fundamentally different entities:
    

In [ ]:
nan is None

In [ ]:
isnan(None)

# Numerical Methods

- In e.g. simulation models or quantiative analysis we typically repeatedly update numerical values inside long loops.

- Programs such as these implement *numerical algorithms*.

- It is very easy to introduce bugs into code like this.



# Numerical stability

- The round-off error associated with a result can be compounded in a loop.

- If the error increases as we go round the loop, we say the algorithm is
numerically *unstable*.

- Mathematicians design numerically stable algorithms using [numerical analysis](https://en.wikipedia.org/wiki/Numerical_analysis).



# Catastrophic Cancellation

- Suppose we have two real values $x$ and $y = x + \epsilon$.

- $\epsilon$ is very small and $x$ is very large.

- $x$ has an _exact_ floating point representation

- However, because of lack of precision $x$ and $y$ have the same floating
point representation.

  - i.e. they are represented as the same sequence of 64-bits


  
# Relative and absolute error

- The absolute error $r = x - y = (x + \epsilon) - x$.

- The relative error is $R = r/x = (x + \epsilon)/x - 1 = \epsilon/x$.

- So the relative error is tiny because $x$ is large relative to $\epsilon$.

- Ok, but how about the error in computing the _difference_ between these values.

- If we perform this in floating-point we will get 0.

- The correct value is $(x + \epsilon) - x = \epsilon$.

- The relative error is infinite!


# Catastrophic Cancellation in Python


In [ ]:
x = 3.141592653589793
x

In [ ]:
y = 6.022e23
x = (x + y) - y

In [ ]:
x


- Avoid subtracting two nearly-equal numbers.

- Especially in a loop!

- Better-yet use a well-validated existing implementation in the form of a numerical library.


# Importing numpy


- Functions for numerical computiing are provided by a separate _module_ called [`numpy`](http://www.numpy.org/).  

- Before we use the numpy module we must import it.

- By convention, we import `numpy` using the alias `np`.

- Once we have done this we can prefix the functions in the numpy library using the prefix `np.`

In [ ]:
import numpy as np

- We can now use the functions defined in this package by prefixing them with `np`.  


# Arrays

- Arrays represent a collection of values.

- In contrast to lists:
    - arrays typically have a *fixed length*
        - they can be resized, but this involves an expensive copying process.
    - and all values in the array are of the *same type*.
        - typically we store floating-point values.

- Like lists:
    - arrays are *mutable*;
    - we can change the elements of an existing array.


# Arrays in `numpy`

    
- Arrays are provided by the `numpy` module.

- The function `array()` creates an array given a list.

In [ ]:
import numpy as np
x = np.array([0, 1, 2, 3, 4])
x

# Array indexing

- We can index an array just like a list

In [ ]:
x[4]

In [ ]:
x[4] = 2
x

# Arrays are not lists

- Although this looks a bit like a list of numbers, it is a fundamentally different type of value:

In [ ]:
type(x)

- For example, we cannot append to the array:

In [ ]:
x.append(5)

# Functions over arrays

- When we use arithmetic operators on arrays, we create a new array with the result of applying the operator to each element.

In [ ]:
y = x * 2
y

- The same goes for numerical functions:

In [ ]:
x = np.array([-1, 2, 3, -4])
y = abs(x)
y

# Vectorized functions

- Note that not every function automatically works with arrays.

- Functions that have been written to work with arrays of numbers are called *vectorized* functions.

- Most of the functions in `numpy` are already vectorized.

- You can create a vectorized version of any other function using the higher-order function `numpy.vectorize()`.

# `vectorize` example

In [ ]:
def myfunc(x):
    if x >= 0.5:
        return x
    else:
        return 0.0
    
fv = np.vectorize(myfunc)

In [ ]:
x = np.arange(0, 1, 0.1)
x

In [ ]:
fv(x)

# Populating Arrays

- To populate an array with a range of values we use the `np.arange()` function:


In [ ]:
x = np.arange(0, 10)
print(x)

- We can also use floating point increments.


In [ ]:
x = np.arange(0, 1, 0.1)
print(x)

# Basic Plotting

- We will use a module called `matplotlib` to plot some simple graphs.

- This module has a nested module called `pyplot`.

- By convention we import this with the alias `plt`.

- This module provides functions which are very similar to [MATLAB plotting commands](http://uk.mathworks.com/help/matlab/ref/plot.html).


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

y = x*2 + 5
plt.plot(x, y)


# Plotting a sine curve

In [ ]:
from numpy import pi, sin

x = np.arange(0, 2*pi, 0.01)
y = sin(x)
plt.plot(x, y)

# Multi-dimensional data

- Numpy arrays can hold multi-dimensional data.

- To create a multi-dimensional array, we can pass a list of lists to the `array()` function:

In [ ]:
import numpy as np

x = np.array([[1,2], [3,4]])
x

# Arrays containing arrays

- A multi-dimensional array is an array of an arrays.

- The outer array holds the rows.

- Each row is itself an array:

In [ ]:
x[0]

In [ ]:
x[1]

- So the element in the second row, and first column is:

In [ ]:
x[1][0]

# Matrices

- We can create a matrix from a multi-dimensional array.

In [ ]:
M = np.matrix(x)
M

# Plotting multi-dimensional with matrices

- If we supply a matrix to `plot()` then it will plot the y-values taken from the *columns* of the matrix (notice the transpose in the example below).

In [ ]:
from numpy import pi, sin, cos

x = np.arange(0, 2*pi, 0.01)
y = sin(x)
ax = plt.plot(x, np.matrix([sin(x), cos(x)]).T)

# Performance 

- When we use `numpy` matrices in Python the corresponding functions are linked with libraries written in C and FORTRAN.

- For example, see the [BLAS (Basic Linear Algebra Subprograms) library](http://www.netlib.org/blas/).

- These libraries are very fast, and can be configured so that operations are performed in parallel on multiple CPU cores, or GPU hardware.


# Matrix Operators

- Once we have a matrix, we can perform matrix computations.

- To compute the [transpose](http://mathworld.wolfram.com/MatrixTranspose.html) and [inverse](http://mathworld.wolfram.com/MatrixInverse.html) use the `T` and `I` attributes:

To compute the transpose $M^{T}$

In [ ]:
M.T


To compute the inverse $M^{-1}$

In [ ]:
M.I

# Matrix Dimensions

- The total number of elements, and the dimensions of the array:

In [ ]:
M.size

In [ ]:
M.shape

In [ ]:
len(M.shape)

# Creating Matrices from strings

- We can also create arrays directly from strings, which saves some typing:

In [ ]:
I2 = np.matrix('2 0; 0 2')
I2

- The semicolon starts a new row.

# Matrix Multiplication

Now that we have two matrices, we can perform [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication):

In [ ]:
M * I2

# Matrix Indexing


- We can [index and slice matrices](http://docs.scipy.org/doc/numpy/user/basics.indexing.html) using the same syntax as lists.

In [ ]:
M[:,1]

# Slices are references

- If we use this is an assignment, we create a *reference* to the sliced elements, *not* a copy.

In [ ]:
V = M[:,1]  # This does not make a copy of the elements!
V

In [ ]:
M[0,1] = -2
V

# Copying matrices and vectors

- To copy a matrix, or a slice of its elements, use the function `np.copy()`:



In [ ]:
M = np.matrix('1 2; 3 4')
V = np.copy(M[:,1])  # This does copy the elements.
V

In [ ]:
M[0,1] = -2
V

# Sums

One way we _could_ sum a vector or matrix is to use a `for` loop.

In [ ]:
vector = np.arange(0.0, 100.0, 10.0)
vector


In [ ]:
result = 0.0
for x in vector:
    result = result + x
result

- This is not the most _efficient_ way to compute a sum.

# Efficient sums

- Instead of using a `for` loop, we can use a numpy function `sum()`.

- This function is written in the C language, and is very fast.


In [ ]:
vector = np.array([0, 1, 2, 3, 4])
print(np.sum(vector))

# Summing rows and columns

- When dealing with multi-dimensional data, the 'sum()' function has a named-argument `axis` which allows us to specify whether to sum along, each rows or columns.


In [ ]:
matrix = np.matrix('1 2 3; 4 5 6; 7 8 9')
print(matrix)

- To sum along rows:

In [ ]:
np.sum(matrix, axis=0)

- To sum along columns:

In [ ]:
np.sum(matrix, axis=1)

# Cumulative sums

- Suppose we want to compute $y_n = \sum_{i=1}^{n} x_i$ where $\mathbf{x}$ is a vector.


In [ ]:
import numpy as np
x = np.array([0, 1, 2, 3, 4])
y = np.cumsum(x)
print(y)

# Cumulative sums along rows and columns


In [ ]:
x = np.matrix('1 2 3; 4 5 6; 7 8 9')
print(x)

In [ ]:
y = np.cumsum(x)
np.cumsum(x, axis=0)

In [ ]:
np.cumsum(x, axis=1)

# Cumulative products

- Similarly we can compute $y_n = \Pi_{i=1}^{n} x_i$ using `cumprod()`:


In [ ]:
import numpy as np
x = np.array([1, 2, 3, 4, 5])
np.cumprod(x)

- We can compute cummulative products along rows and columns using the `axis` parameter, just as with the `cumsum()` example.

# Generating (pseudo) random numbers

- The nested module `numpy.random` contains functions for generating random numbers from different probability distributions.


In [ ]:
from numpy.random import normal, uniform, exponential, randint

- Suppose that we have a random variable $\epsilon \sim N(0, 1)$.

- In Python we can draw from this distribution like so:

In [ ]:
epsilon = normal()
print(epsilon)

- If we execute another call to the function, we will make a _new_ draw from the distribution:

In [ ]:
epsilon = normal()
print(epsilon)

# Pseudo-random numbers

- Strictly speaking, these are not random numbers.

- They rely on an initial state value called the *seed*.

- If we know the seed, then we can predict with total accuracy the rest of the sequence, given any "random" number.

- Nevertheless, statistically they behave like independently and identically-distributed values.
    - Statistical tests for correlation and auto-correlation give insignificant results.

- For this reason they called *pseudo*-random numbers.

- The algorithms for generating them are called Pseudo-Random Number Generators (PRNGs).

- Some applications, such as cryptography, require genuinely unpredictable sequences.
    - never use a standard PRNG for these applications!

# Managing seed values

- In some applications we need to reliably reproduce the same sequence of pseudo-random numbers that were used.

- We can specify the seed value at the beginning of execution to achieve this.

- Use the function `seed()` in the `numpy.random` module.



# Setting the seed

In [ ]:
from numpy.random import seed

seed(5)


In [ ]:
normal()

In [ ]:
normal()

In [ ]:
seed(5)

In [ ]:
normal()

In [ ]:
normal()

# Drawing multiple variates

- To generate more than number, we can specify the `size` parameter:

In [ ]:
normal(size=10)

- If you are generating very many variates, this will be *much* faster than using a for loop

- We can also specify more than one dimension:


In [ ]:
normal(size=(5,5))

# Histograms

- We can plot a histograms of randomly-distributed data using the `hist()` function from matplotlib:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

data = normal(size=10000)
ax = plt.hist(data)

# Computing histograms as matrices

- The function `histogram()` in the `numpy` module will count frequencies into bins and return the result as a 2-dimensional array.

In [ ]:
import numpy as np
np.histogram(data)

# Summary statistics

- We can compute the summary statistics of a sample of values using the numpy functions `mean()` and `var()` to compute the sample mean $\bar{X}$ and sample [variance](https://en.wikipedia.org/wiki/Variance) $\sigma_{X}^2$ .


In [ ]:
np.mean(data)

In [ ]:
np.var(data)

- These functions also have an `axis` parameter to compute mean and variances of columns or rows of a multi-dimensional data-set.

# Summary statistics with `nan` values

- If the data contains `nan` values, then the summary statistics will also be `nan`.



In [ ]:
from numpy import nan
import numpy as np
data = np.array([1, 2, 3, 4, nan])
np.mean(data)

- To omit `nan` values from the calculation, use the functions `nanmean()` and `nanvar()`:

In [ ]:
np.nanmean(data)

# Discrete random numbers

- The `randint()` function in `numpy.random` can be used to draw from a uniform discrete probability distribution.

- It takes two parameters: the low value (inclusive), and the high value (exclusive).

- So to simulate one roll of a die, we would use the following Python code.


In [ ]:
die_roll = randint(0, 6) + 1
die_roll

- Just as with the `normal()` function, we can generate an entire sequence of values.

- To simulate a [Bernoulli process](https://en.wikipedia.org/wiki/Bernoulli_process) with $n=20$ trials:

In [ ]:
bernoulli_trials = randint(0, 2, size = 20)
bernoulli_trials

### Acknowledgements

The earlier sections of this notebook were adapted from [an article on floating-point numbers](http://steve.hollasch.net/cgindex/coding/ieeefloat.html) written by [Steve Hollasch](http://steve.hollasch.net/).